In [2]:
import tkinter as tk
from tkinter import messagebox, font
import subprocess
import json
import logging
from pathlib import Path
import sys
import random

# 로깅 설정
log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)
logging.basicConfig(filename=log_dir / 'script_launcher.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def load_config():
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            config = json.load(f)
            if 'scripts' not in config:
                raise ValueError("Config file must contain 'scripts' key")
            for script in config['scripts']:
                if 'name' not in script or 'path' not in script:
                    raise ValueError("Each script must have 'name' and 'path'")
            return config
    except Exception as e:
        logging.exception("Failed to load config")
        messagebox.showerror("Error", f"설정 파일을 불러오는데 실패했습니다: {e}")
        return None

def run_script(script_path):
    try:
        if script_path.endswith(".ipynb"):
            subprocess.Popen(['jupyter', 'nbconvert', '--to', 'notebook', '--execute', script_path])
        else:
            subprocess.Popen([sys.executable, script_path])
        logging.info(f"Script executed: {script_path}")
    except Exception as e:
        logging.exception(f"Failed to run {script_path}")
        messagebox.showerror("Error", f"스크립트 실행 실패: {Path(script_path).name}\nError: {str(e)}")

def on_exit():
    if messagebox.askokcancel("종료", "프로그램을 종료하시겠습니까?"):
        logging.info("Application closed")
        root.destroy()

def random_color():
    return f'#{random.randint(0, 0xFFFFFF):06x}'

def create_button(script, row, col):
    return tk.Button(root, text=script['name'], 
                     command=lambda path=script['path']: run_script(path),
                     font=button_font, bg=random_color(), fg='white',
                     activebackground=random_color(), activeforeground='white',
                     wraplength=150,  # 텍스트 줄바꿈
                     width=20, height=3).grid(row=row, column=col, padx=10, pady=10)

root = tk.Tk()
root.title("스크립트 실행기")
root.geometry("400x500")  # 윈도우 크기 설정
root.configure(padx=10, pady=10)  # 여백 설정

# 폰트 설정
button_font = font.Font(size=10, weight="bold")

# 설정 불러오기
config = load_config()
 
if config:
    # 2열로 버튼 배치
    max_columns = 2
    for i, script in enumerate(config['scripts']):
        row = i // max_columns
        col = i % max_columns
        create_button(script, row, col)
        
# 종료 버튼을 마지막 행에 전체 열을 합쳐 배치
btn_exit = tk.Button(root, text="종료", command=on_exit,
                     font=button_font, bg='#f44336', fg='white',
                     activebackground='#d32f2f', activeforeground='white',
                     width=20, height=3)
btn_exit.grid(row=(len(config['scripts']) // max_columns) + 1, column=0, columnspan=2, pady=20)

root.protocol("WM_DELETE_WINDOW", on_exit)  # 윈도우 닫기 버튼에 on_exit 바인딩

logging.info("Application started")
root.mainloop()
